In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
# Set the path to your dataset

data_dir = "F:\\Plant Disease Classification\\mini\\train"

In [3]:
# Set the input size of the model

input_size = (224, 224)

In [4]:
# Set the batch size and number of classes

batch_size = 32
num_classes = 38

In [5]:
# Preprocess the input images

data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [6]:
# Load and augment the training data

train_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 6080 images belonging to 38 classes.


In [7]:
# Load the validation data

validation_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1520 images belonging to 38 classes.


In [8]:
# Load the pre-trained ResNet-50 model without the top (classification) layer

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_size + (3,))

In [9]:
# Freeze the pre-trained layers

for layer in resnet_model.layers:
    layer.trainable = False

In [10]:
# Create the final model

resnet_classifier = Sequential()
resnet_classifier.add(resnet_model)
resnet_classifier.add(GlobalAveragePooling2D())
resnet_classifier.add(Dense(512, activation='relu'))
resnet_classifier.add(Dense(num_classes, activation='softmax'))

In [11]:
# Compile the model

resnet_classifier.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the ResNet-50 model

history = resnet_classifier.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=1
)

190/190 [==============================] - 590s 3s/step - loss: 3.3933 - accuracy: 0.0765 - val_loss: 3.2994 - val_accuracy: 0.0911


In [16]:
# Get the validation accuracy from the history object

validation_accuracy = history.history['val_accuracy']

In [18]:
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: [0.09109042584896088]


In [24]:
# Set the path to your test dataset directory

test_dir = "F:\\Plant Disease Classification\mini\\test"

# Load the test data

test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 380 images belonging to 38 classes.


In [26]:
# Evaluate the VGG16 model on the test data

evaluation = resnet_classifier.evaluate(test_generator)

# Get the evaluation score (accuracy)

evaluation_score = evaluation[1]  # Index 1 corresponds to the accuracy value

12/12 [==============================] - 52s 4s/step - loss: 3.2766 - accuracy: 0.1105


In [27]:
print("Evaluation Accuracy:", evaluation_score)

Evaluation Accuracy: 0.11052631586790085
